# Police Force Responses by Crime

In [1]:
#Dependencies
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd

In [2]:
#Import csv file
police_data = pd.read_csv("Resources/Police_Use_of_Force.csv")
police_data.head()

,X,Y,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,Is911Call,PrimaryOffense,SubjectInjury,ForceReportNumber,...,TotalCityCallsForYear,TotalPrecinctCallsForYear,TotalNeighborhoodCallsForYear,CenterGBSID,CenterLatitude,CenterLongitude,CenterX,CenterY,DateAdded,OBJECTID
0,-93.299256,44.980557,19194596,08-007748,2008/01/08 14:20:21+00,Disturbance,No,DISCON,NaN,1,...,322402,80434.0,2514,15824,44.980557,-93.299256,-10386025.66,5618461.049,2021/02/11 08:18:54+00,1
1,-93.273109,44.979187,19194597,08-019047,2008/01/19 23:47:58+00,On Site,No,NOSMOK,NaN,1,...,322402,46998.0,23458,17239,44.979187,-93.273109,-10383115.01,5618245.449,2021/02/11 08:18:54+00,2
2,-93.311817,45.006028,19194598,08-034409,2008/02/04 21:21:49+00,Property Damage Accident,Yes,ASLT2,NaN,1,...,322402,80434.0,9486,10909,45.006028,-93.311817,-10387423.95,5622470.553,2021/02/11 08:18:54+00,3
3,-93.288860,45.043001,19194599,08-043573,2008/02/14 22:03:15+00,Unwanted Person,Yes,DAMPRP,NaN,2,...,322402,80434.0,3070,17338,45.043001,-93.288860,-10384868.44,5628293.683,2021/02/11 08:18:54+00,4
4,-93.271963,44.981655,19194600,08-052540,2008/02/24 02:25:39+00,Fight,No,OBSTRU,No,3,...,322402,46998.0,23458,17120,44.981655,-93.271963,-10382987.47,5618633.899,2021/02/11 08:18:54+00,5


In [3]:
police_data.columns

Index(['X', 'Y', 'PoliceUseOfForceID', 'CaseNumber', 'ResponseDate', 'Problem',
       'Is911Call', 'PrimaryOffense', 'SubjectInjury', 'ForceReportNumber',
       'SubjectRole', 'SubjectRoleNumber', 'ForceType', 'ForceTypeAction',
       'Race', 'Sex', 'EventAge', 'TypeOfResistance', 'Precinct',
       'Neighborhood', 'TotalCityCallsForYear', 'TotalPrecinctCallsForYear',
       'TotalNeighborhoodCallsForYear', 'CenterGBSID', 'CenterLatitude',
       'CenterLongitude', 'CenterX', 'CenterY', 'DateAdded', 'OBJECTID'],
      dtype='object')

In [14]:
police_data_df = police_data.loc[:, ["PoliceUseOfForceID", "CaseNumber", "ResponseDate", "Problem", "SubjectInjury", "ForceType",
                                    "ForceTypeAction", "EventAge", "TypeOfResistance", "Precinct", "Neighborhood"]]
police_data_df

,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,SubjectInjury,ForceType,ForceTypeAction,EventAge,TypeOfResistance,Precinct,Neighborhood
0,19194596,08-007748,2008/01/08 14:20:21+00,Disturbance,NaN,Bodily Force,Push Away,13.0,Tensed,4,Harrison
1,19194597,08-019047,2008/01/19 23:47:58+00,On Site,NaN,Bodily Force,Push Away,27.0,Tensed,1,Downtown West
2,19194598,08-034409,2008/02/04 21:21:49+00,Property Damage Accident,NaN,Bodily Force,Joint Lock,17.0,Verbal Non-Compliance,4,Willard - Hay
3,19194599,08-043573,2008/02/14 22:03:15+00,Unwanted Person,NaN,Bodily Force,Body Weight to Pin,18.0,Tensed,4,Lind - Bohanon
4,19194600,08-052540,2008/02/24 02:25:39+00,Fight,No,Bodily Force,Knees,28.0,Commission of Crime,1,Downtown West
...,...,...,...,...,...,...,...,...,...,...,...
32030,19226626,09-Cooper,2009/01/01 00:00:00+00,NaN,NaN,NaN,0,NaN,NaN,NaN,Cooper
32031,19226627,08-Bottineau,2008/01/01 00:00:00+00,NaN,NaN,NaN,0,NaN,NaN,NaN,Bottineau
32032,19226628,09-Loring Park,2009/01/01 00:00:00+00,NaN,NaN,NaN,0,NaN,NaN,NaN,Loring Park
32033,19226629,14-Cooper,2014/01/01 00:00:00+00,NaN,NaN,NaN,0,NaN,NaN,NaN,Cooper


In [15]:
police_data_df.dtypes

PoliceUseOfForceID      int64
CaseNumber             object
ResponseDate           object
Problem                object
SubjectInjury          object
ForceType              object
ForceTypeAction        object
EventAge              float64
TypeOfResistance       object
Precinct               object
Neighborhood           object
dtype: object

In [21]:
police_data_df = police_data_df.dropna(how='any')
police_data_df

,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,SubjectInjury,ForceType,ForceTypeAction,EventAge,TypeOfResistance,Precinct,Neighborhood
4,19194600,08-052540,2008/02/24 02:25:39+00,Fight,No,Bodily Force,Knees,28.0,Commission of Crime,1,Downtown West
5,19194601,08-071686,2008/03/14 01:24:28+00,Emotionally Disturb Person,No,Bodily Force,Body Weight to Pin,21.0,Fled on Foot,3,Ventura Village
6,19194602,08-081163,2008/03/23 04:05:00+00,Suspicious Vehicle,Yes,Bodily Force,Punches,15.0,Fled in Vehicle,4,Willard - Hay
7,19194603,08-088888,2008/03/30 20:42:05+00,Traffic Law Enforcement,Yes,Bodily Force,Punches,36.0,Fled on Foot,4,Hawthorne
9,19194605,08-103121,2008/04/13 00:19:43+00,Burglary Dwlng In Progress,Yes,Bodily Force,Slaps,19.0,Fled on Foot,4,Shingle Creek
...,...,...,...,...,...,...,...,...,...,...,...
30817,19225413,20-238499,2020/09/14 17:33:22+00,Emotionally Disturb Person,No,Bodily Force,Joint Lock,28.0,Tensed ...,2,Marcy Holmes
30822,19225418,20-255690,2020/10/04 02:23:49+00,Sound of Shots Fired,Yes,Bodily Force,Takedown,27.0,Fled on Foot ...,1,Downtown West
30825,19225421,20-275777,2020/10/28 22:12:02+00,Domestic Abuse-In Progress,No,Bodily Force,Body Weight Pin,24.0,Other ...,5,Windom
30827,19225423,20-281708,2020/11/04 20:54:28+00,Domestic Abuse Report Only,No,Bodily Force,Body Weight Pin,27.0,Other ...,2,Holland


In [22]:
police_data_df.drop_duplicates(subset=['ResponseDate'])

,PoliceUseOfForceID,CaseNumber,ResponseDate,Problem,SubjectInjury,ForceType,ForceTypeAction,EventAge,TypeOfResistance,Precinct,Neighborhood
4,19194600,08-052540,2008/02/24 02:25:39+00,Fight,No,Bodily Force,Knees,28.0,Commission of Crime,1,Downtown West
5,19194601,08-071686,2008/03/14 01:24:28+00,Emotionally Disturb Person,No,Bodily Force,Body Weight to Pin,21.0,Fled on Foot,3,Ventura Village
6,19194602,08-081163,2008/03/23 04:05:00+00,Suspicious Vehicle,Yes,Bodily Force,Punches,15.0,Fled in Vehicle,4,Willard - Hay
7,19194603,08-088888,2008/03/30 20:42:05+00,Traffic Law Enforcement,Yes,Bodily Force,Punches,36.0,Fled on Foot,4,Hawthorne
9,19194605,08-103121,2008/04/13 00:19:43+00,Burglary Dwlng In Progress,Yes,Bodily Force,Slaps,19.0,Fled on Foot,4,Shingle Creek
...,...,...,...,...,...,...,...,...,...,...,...
30636,19225232,18-031583,2018/01/31 07:42:53+00,Domestic Abuse-In Progress,Yes,Bodily Force,Push Away,23.0,Tensed,3,East Phillips
30643,19225239,18-121861,2018/04/18 18:52:38+00,Domestic with Weapons,Yes,Bodily Force,Body Weight to Pin,25.0,Tensed,4,Cleveland
30644,19225240,18-145488,2018/05/07 02:43:24+00,Suspicious Person,No,Bodily Force,Body Weight to Pin,30.0,Tensed,5,Lowry Hill East
30816,19225412,20-229068,2020/09/03 09:58:16+00,Theft,No,Maximal Restraint Technique,Side Recovery Position,40.0,Other ...,1,North Loop


# 1. What incidents result the most in police use of force? (Sam)

In [ ]:
#b

# 2. What type of force is used the most? (Ainash)

# 3. Which is the most popular type of resistance? (Humera)

# 4. Which neighborhood in Minneapolis see’s the highest frequency of police force? (Mike Hills)

# 5. Is the use of police force in Minneapolis going up or down? (Mike F)

# 6. Perform a linear regression on use of force between 2015 and 2019 and make a prediction about use of force in 2020 for the city of Minneapolis.  Compare this prediction to actual use of force outcomes in 2020.